In [1]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from string import punctuation
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
#nltk.download('perluniprops')
from nltk import ngrams
from itertools import chain
from wordcloud import WordCloud

import csv
import re
import pickle

### Fetch the data

In [3]:
with open('tokenized_reviews.pickle', 'rb') as handle:
    df = pickle.load(handle)

df.head()

,Score,Label,Text
0,5,1,"[i, have, bought, several, of, the, vitality, ..."
1,1,0,"[product, arrived, labeled, as, jumbo, salted,..."
2,4,1,"[this, is, a, confection, that, has, been, aro..."
3,2,0,"[if, you, are, looking, for, the, secret, ingr..."
4,5,1,"[great, taffy, at, a, great, price, ., there, ..."


## Create a model

In [ ]:
# Add your import statements here
from keras import Input, Model
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, Embedding
from keras.layers.recurrent import LSTM
from keras.callbacks import TensorBoard, EarlyStopping
from keras.optimizers import Adam, RMSprop

def create_model(input_n, X_seq_len, output_n, Y_seq_len, hidden_dim, embedding_dim):
    """ Define a keras sequence-to-sequence model. 
    
    Arguments:
    input_n - integer, the number of inputs for the network (the length of a one-hot vector from `X`)
    X_seq_len - integer, the length of a sequence from `X`. Should be constant and you made sure by using padding
    output_n - integer, the number of outputs for the network (the length of a one-hot vector from `Y`)
    Y_seq_len - integer, the length of a sequence from `Y`. Should be constant and you made sure by using padding
    hidden_dim - integer, number of units in the LSTM's memory cell.
    embedding_dim - output dimension of the embedding layer.
    
    Returns:
    The compiled keras model
    
    """
    # Input and embedding layers
    input_layer = Input(shape=(None,1))
    
    # Create the encoder LSTM.
    # correct number of units?
    encoder_LSTM = LSTM(units=hidden_dim, activation='tanh', recurrent_activation='hard_sigmoid', return_state=True)
    encoder_out1, encoder_out2, encoder_out3 = encoder_LSTM(encoder_embedding_layer)
    encoder_states = [encoder_out2, encoder_out3]
    
    # Null input (should be fed with zeros) and repeating it the same number of times as there are words in the 
    # target sentence
    null_input = Input(shape=[1])
    repeated_null = RepeatVector(Y_seq_len)(null_input)
    
    # Create the decoder LSTM (feed it with the memory of the encoder network). Call it `decoder_layer`
    decoder_LSTM = LSTM(return_sequences=True, units=hidden_dim)#(repeated_null)
    decoder_layer = decoder_LSTM(repeated_null,initial_state=encoder_states)
    
    # Add a fully connected layer and a softmax to the outputs of the decoder
    decoder_fully_connected = TimeDistributed(Dense(output_n))(decoder_layer)
    decoder_softmax = Activation('softmax')(decoder_fully_connected)
    
    # Create final model and compile it
    model = Model([encoder_input_layer, null_input], decoder_softmax)
    
    # Compile the model. Use a loss function, optimizer, and metrics of your choice
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Add these arguments to the model for convenience
    model.hidden_dim = hidden_dim
    model.embedding_dim = embedding_dim
    
    return model

### Create an embedding layer

In [14]:
from gensim.models import word2vec

# Load Google's pre-trained Word2Vec model.
embedding_model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

In [15]:
embedding_model.most_similar(['girl', 'father'], ['boy'], topn=3)

[('mother', 0.831214427947998),
 ('daughter', 0.8000643253326416),
 ('husband', 0.769158124923706)]